In [1]:
import cv2
import mediapipe as mp
import numpy as np
import schedule
from time import time
from time import localtime
import serial
import math

# 인식할 수 있는 손 개수
max_num_hands = 2
# 9가지의 제스처, 제스처 데이터는 손가락 관절의 각도와 각각의 라벨
gesture = {
    0:'tvon', 1:'tvoff', 2:'chup', 3:'chdown', 4:'volup', 5:'voldown',
    6:'fanon', 7:'fanoff', 8:'click'
} 

# Arduino 연결
ser = serial.Serial(port='COM9',
                    baudrate=115200,
   )

channel = 1
channel2 = 0
volume = 0
volume2 = 0
tv_st = 'OFF'
win_st = ['CLOSE', 'CLOSE']
led_st = ['OFF','OFF','OFF','OFF']
sc_st = 'TV'
sc_sts = 'first'
fan_st = 'OFF'
color1 = (255, 255, 255) 
finger1 = 0
finger2 = 0
A = ""
B = ""
Trans = ""

# color 설정
blue_color = (255, 0, 0)
red_color = (0, 0, 255)
black_color = (0, 0, 0)

if channel2 % 10 == 0:
    channel  = channel + 1

def ch_up():
    global channel
    global channel2
    channel2 = channel2 + 1
    
    if channel2 % 13 == 0:
        channel = channel + 1

    if channel > 5:
        channel = 5
        
    return channel
    
def ch_down():
    global channel 
    global channel2
    channel2 = channel2 + 1
    
    if channel2 % 13 == 0:
        channel = channel - 1
    
    if channel < 1:
        channel = 1
    
    return channel


def vol_up():
    global volume
    global volume2
    volume2 = volume2 + 1
    
    if volume2 % 5 == 0:
        volume = volume + 1
    
    if volume >= 100:
        volume = 100
    return volume

def vol_down():
    global volume 
    global volume2
    volume2 = volume2 + 1
    
    if volume2 % 5 == 0:
        volume = volume - 1
        
    if volume < 0:
        volume = 0
    return volume

def overlay(image, x, y, w, h, overlay_image): # 대상 이미지 (3채널), x, y 좌표, width, height, 덮어씌울 이미지 (4채널)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 255 로 나누면 0 ~ 1 사이의 값 (1: 불투명, 0: 완전)
    # (255, 255)  ->  (1, 1)
    # (255, 0)        (1, 0)
    
    # 1 - mask_image ?
    # (0, 0)
    # (0, 1)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# 이미지 로드
imgfile = '/Users/cyoun/Project/image/help.png'
help_image = cv2.imread(imgfile, cv2.IMREAD_UNCHANGED)
helph, helpw, _ = help_image.shape

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 제스처 인식 모델
file = np.genfromtxt('data/gestures_train.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32) # float32의 형태로 저장
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create() # KNN 알고리즘 객체 생성
knn.train(angle, cv2.ml.ROW_SAMPLE, label) # KNN 알고리즘 학습 / 하나의 데이터가 한 행으로 구성됨

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        continue
        
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    h, w, _ = img.shape
    times = time()
    tm = localtime(times)


        
    # 첫 화면
    if (sc_sts == 'first'):
        cv2.putText(img, 'START', (w//2 - 140, h//2 + 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
        cv2.putText(img, 'EXIT', (w//2 + 50, h//2 + 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
        cv2.putText(img, 'UKSUNG',
                            (w//2 - 200, h//2-30),
                            cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 3, color1, 3, cv2.LINE_AA)
        cv2.putText(img, 'Click to start',
                            (w//2 - 120, h//2 + 30),
                            cv2.FONT_ITALIC, 1, (51, 102, 153), 1, cv2.LINE_AA)
        if (30 < finger1 < 40):
            if(64 < finger2 <= 68):
                sc_sts = 'twice'
                sc_st = 'start'
        if (62 < finger1 < 66):
            if(64 < finger2 <= 68):
                break
    
    # 두 번째 화면
    if (sc_sts == 'twice'):
        
        if (10 <= finger1 < 15):
            if (5 < finger2 < 15):
                sc_st = 'help'
        if (35 <= finger1 < 40):
            if(5 < finger2 < 15):
                sc_st = 'TV'
 
        if (65 <= finger1 < 70):
            if(5 < finger2 < 15):
                sc_st = 'EX'
            
        if (90 <= finger1 < 95):
            if(5 < finger2 < 15):
                sc_st = 'exit'
               
        # HELP 버튼
        img = cv2.rectangle(img, (20,20), (100, 70), black_color, -1)
        cv2.putText(img, 'HELP', (20, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
        
        # TV 버튼 
        img = cv2.rectangle(img, (180,20), (260, 70), black_color, -1)
        cv2.putText(img, 'TV', (205, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)

        # 전자 버튼
        img = cv2.rectangle(img, (380,20), (460, 70), black_color, -1)
        cv2.putText(img, 'EX', (400, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
    
        # 종료 버튼
        img = cv2.rectangle(img, (540,20), (620, 70), black_color, -1)
        cv2.putText(img, 'BACK', (540, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
        
        if (sc_st == 'TV'):
            cv2.putText(
                img, f'TV : {tv_st.upper()}', (w//6 - 80, h//3),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(
                img, f'CHANNEL : {channel}', (w//6 - 80, h//3 + 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(
                img, f'VOLUME : {volume}', (w//6 - 80, h//3 + 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            if tv_st =='ON':
                B = "p"
                
            if tv_st =='OFF':
                B = "q"
                
                
            
            # TV 채널
            img = cv2.rectangle(img, (w - 230, h//3 - 10), (w - 150, h//3 + 160), blue_color, -1)
            cv2.putText(img, '+', (w - 200, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            img = cv2.rectangle(img, (w - 230, h//3 + 75), (w - 150, h//3 + 160), blue_color, -1)
            cv2.putText(img, '-', (w - 200, h//3 +145), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, 'ch', (w - 200, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
            
            # TV 음량
            img = cv2.rectangle(img, (w - 110, h//3 - 10), (w - 30, h//3 + 160), blue_color, -1)
            cv2.putText(img, '+', (w - 80, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, '-', (w - 80, h//3 + 145), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, 'vol', (w - 85, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
            
            # TV 채널
            if (64 <= finger1 <= 76):
                if (32 < finger2 < 47):
                    img = cv2.rectangle(img, (w - 230, h//3 - 10), (w - 150, h//3 + 75), red_color, -1)
                    cv2.putText(img, '+', (w - 200, h//3 + 25), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                    cv2.putText(img, 'ch', (w - 200, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
                    if tv_st =='ON':
                        ch_up()
                        
            if (64 <= finger1 <= 76):
                if (47 < finger2 < 66):
                    img = cv2.rectangle(img, (w - 230, h//3 + 75), (w - 150, h//3 + 160), red_color, -1)
                    cv2.putText(img, '-', (w - 200, h//3 +145), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                    cv2.putText(img, 'ch', (w - 200, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
                    if tv_st =='ON':
                        ch_down()
  
                    
            # TV 음량
            if (83 <= finger1 <= 94):
                if (32 < finger2 < 47):
                    img = cv2.rectangle(img, (w - 110, h//3 - 10), (w - 30, h//3 + 75), red_color, -1)
                    cv2.putText(img, '+', (w - 80, h//3 + 25), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                    cv2.putText(img, 'vol', (w - 85, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
                    if tv_st =='ON':
                        vol_up()
                        
            if (83 <= finger1 <= 94):
                if (47 < finger2 < 66):
                    img = cv2.rectangle(img, (w - 110, h//3 + 75), (w - 30, h//3 + 160), red_color, -1)
                    cv2.putText(img, '-', (w - 80, h//3 + 145), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                    cv2.putText(img, 'vol', (w - 85, h//3 + 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,  color1, 2, cv2.LINE_AA)
                    if tv_st =='ON':
                        vol_down()
                       
            if (channel == 1):
                if (tv_st == 'ON'):
                    A = str(volume)
                    Trans = "k" + B + A 
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                if (tv_st == 'OFF'):
                    Trans = "k" + B
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                
            if (channel == 2):
                if (tv_st == 'ON'):
                    A = str(volume)
                    Trans = "l" + B + A
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                if (tv_st == 'OFF'):
                    Trans = "l" + B
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                
            if (channel == 3):
                if (tv_st == 'ON'):
                    A = str(volume)
                    Trans = "m" + B + A
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                if (tv_st == 'OFF'):
                    Trans = "m" + B
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                
            if (channel == 4):
                if (tv_st == 'ON'):
                    A = str(volume)
                    Trans = "n" + B + A
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                if (tv_st == 'OFF'):
                    Trans = "n" + B
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                
            if (channel == 5):
                if (tv_st == 'ON'):
                    A = str(volume)
                    Trans = "o" + B + A
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
            if (tv_st == 'OFF'):
                    Trans = "o" + B
                    Trans = Trans.encode('utf-8')
                    ser.write(Trans)
                
        # LED / PAN ...
        elif (sc_st == 'EX'):
            img = cv2.rectangle(img, (w - 230, h//3 - 10), (w - 150, h//3 + 40), blue_color, -1)
            cv2.putText(img, 'LED1', (w - 225, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            img = cv2.rectangle(img, (w - 110, h//3 - 10), (w - 30, h//3 + 40), blue_color, -1)
            cv2.putText(img, 'LED2', (w - 105, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
    
            img = cv2.rectangle(img, (w - 230, h//3 + 80), (w - 150, h//3 + 130), blue_color, -1)
            cv2.putText(img, 'LED3', (w - 225, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            img = cv2.rectangle(img, (w - 110, h//3 + 80), (w - 30, h//3 + 130), blue_color, -1)
            cv2.putText(img, 'LED4', (w - 105, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            img = cv2.rectangle(img, (w - 230, h//3 + 170), (w - 150, h//3 + 220), blue_color, -1)
            cv2.putText(img, 'WIN1', (w - 225, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            img = cv2.rectangle(img, (w - 110, h//3 + 170), (w - 30, h//3 + 220), blue_color, -1)
            cv2.putText(img, 'WIN2', (w - 105, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            
            
            cv2.putText(img, f'LED 1 : {led_st[0].upper()}', (w//6 - 80, h//3), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, f'LED 2 : {led_st[1].upper()}', (w//6 - 80, h//3 + 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, f'LED 3 : {led_st[2].upper()}', (w//6 - 80, h//3 + 100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, f'LED 4 : {led_st[3].upper()}', (w//6 - 80, h//3 + 150), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            cv2.putText(img, f'FAN  : {fan_st.upper()}', (w//6 - 80, h//3 + 200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                
            if (led_st[0] == 'OFF'):
                img = cv2.rectangle(img, (w - 230, h//3 - 10), (w - 150, h//3 + 40), blue_color, -1)
                cv2.putText(img, 'LED1', (w - 225, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
               # ser.write(b'abcdefghij')
                if(64 <= finger1 <=70):
                    if(32 <= finger2 <=42):
                        led_st[0] = 'ON'
                        ser.write(b'a')
            
            if (led_st[0] == 'ON'):
                img = cv2.rectangle(img, (w - 230, h//3 - 10), (w - 150, h//3 + 40), red_color, -1)
                cv2.putText(img, 'LED1', (w - 225, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
           #     ser.write(b'Abcdefghij')
                if(70 <= finger1 <=76):
                    if(32 <= finger2 <=42):
                        led_st[0] = 'OFF'
                        ser.write(b'b')
                        
                        
            if (led_st[1] == 'OFF'):
                img = cv2.rectangle(img, (w - 110, h//3 - 10), (w - 30, h//3 + 40), blue_color, -1)
                cv2.putText(img, 'LED2', (w - 105, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
           #     ser.write(b'abcdefghij')
                if(83 <= finger1 <=89):
                    if(32 <= finger2 <=42):
                        led_st[1] = 'ON'
                        ser.write(b'c')
            
            if (led_st[1] == 'ON'):
                img = cv2.rectangle(img, (w - 110, h//3 - 10), (w - 30, h//3 + 40), red_color, -1)
                cv2.putText(img, 'LED2', (w - 105, h//3 + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            #    ser.write(b'aBcdefghij')
                if(89 <= finger1 <=94):
                    if(32 <= finger2 <=42):
                        led_st[1] = 'OFF'
                        ser.write(b'd')
                        
            if (led_st[2] == 'OFF'):
                img = cv2.rectangle(img, (w - 230, h//3 + 80), (w - 150, h//3 + 130), blue_color, -1)
                cv2.putText(img, 'LED3', (w - 225, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
            #    ser.write(b'abcdefghij')
                if(64 <= finger1 <=70):
                    if(50 <= finger2 <=60):
                        led_st[2] = 'ON'
                        ser.write(b'e')
            
            if (led_st[2] == 'ON'):
                img = cv2.rectangle(img, (w - 230, h//3 + 80), (w - 150, h//3 + 130), red_color, -1)
                cv2.putText(img, 'LED3', (w - 225, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
             #   ser.write(b'abCdefghij')
                if(70 <= finger1 <=76):
                    if(50 <= finger2 <=60):
                        led_st[2] = 'OFF'
                        ser.write(b'f')
            
            if (led_st[3] == 'OFF'):
                img = cv2.rectangle(img, (w - 110, h//3 + 80), (w - 30, h//3 + 130), blue_color, -1)
                cv2.putText(img, 'LED4', (w - 105, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
               # ser.write(b'abcdefghij')
                if(83 <= finger1 <=89):
                    if(50 <= finger2 <=60):
                        led_st[3] = 'ON'
                        ser.write(b'g')
            
            if (led_st[3] == 'ON'):
                img = cv2.rectangle(img, (w - 110, h//3 + 80), (w - 30, h//3 + 130), red_color, -1)
                cv2.putText(img, 'LED4', (w - 105, h//3 + 115), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
               # ser.write(b'abcDefghij')
                if(89 <= finger1 <=94):
                    if(50 <= finger2 <=60):
                        led_st[3] = 'OFF'
                        ser.write(b'h')
            
            if (win_st[0] == "CLOSE"):
                img = cv2.rectangle(img, (w - 230, h//3 + 170), (w - 150, h//3 + 220), blue_color, -1)
                cv2.putText(img, 'WIN1', (w - 225, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                if (64 <= finger1 <= 70):
                    if(70 <= finger2 <= 78):
                        win_st[0] = "OPEN"
                        ser.write(b's')    
                
            if (win_st[0] == "OPEN"):
                img = cv2.rectangle(img, (w - 230, h//3 + 170), (w - 150, h//3 + 220), red_color, -1)
                cv2.putText(img, 'WIN1', (w - 225, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                if (70 <= finger1 <= 76):
                    if(70 <= finger2 <= 78):
                        win_st[0] = "CLOSE"
                        ser.write(b't')
                    
            if (win_st[1] == "OPEN"):
                img = cv2.rectangle(img, (w - 110, h//3 + 170), (w - 30, h//3 + 220), red_color, -1)
                cv2.putText(img, 'WIN2', (w - 105, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                if (89 <= finger1 <= 94):
                    if(70 <= finger2 <= 78):
                        win_st[1] = "CLOSE"
                        ser.write(b'v')
                
            if (win_st[1] == "CLOSE"):
                img = cv2.rectangle(img, (w - 110, h//3 + 170), (w - 30, h//3 + 220), blue_color, -1)
                cv2.putText(img, 'WIN2', (w - 105, h//3 + 205), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color1, 2, cv2.LINE_AA)
                if (83 <= finger1 <= 89):
                    if(70 <= finger2 <= 78):
                        win_st[1] = "OPEN"
                        ser.write(b'u')

        # exit
        elif (sc_st == 'exit'):
              sc_sts = 'first'
        # help        
        elif (sc_st == 'help'):
              overlay(img, w//2, h//2, 220, 160, help_image)


    
    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]
            finger1 = int(joint[8][0] * 100) # x 좌표
            finger2 = int(joint[8][1] * 100) # y 좌표
            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree
            
            cv2.putText(
                    img, text='f1=%d f2=%d' % (finger1, finger2), org=(10, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                    color=255, thickness=3)
            
            # Inference gesture
            data = np.array([angle], dtype=np.float32)
            # 모든 픽셀에 대해 몇 번 클래스 인지 계산 
            # result : 각 입력 샘플에 대한 예측 결과를 저장한 행렬
            # neighbours : 예측에 사용된 k개의 최근접 이웃 클래스 정보 행렬
            # dist : 입력 벡터와 예측에 사용된 K개의 최근접 이웃과의 거리를 저장한 행렬
            # ret : 입력 벡터가 하나인 경우에 대한 응답
             
            ret, results, neighbours, dist = knn.findNearest(data, 3) # 사용할 최근접 이웃 개수 
            idx = int(results[0][0])
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            this_action = '?'
            
            if (sc_st == 'TV'):
                if idx == 0:
                    tv_st = 'ON'
                    this_action = 'TVON'

                if idx == 1:
                    tv_st = 'OFF'
                    this_action = 'TVOFF'

                if idx == 2:
                    this_action = 'CHUP' 
                    if tv_st =='ON':
                        ch_up()

                if idx == 3:
                    this_action = 'CHDOWN'
                    if tv_st == 'ON':
                        ch_down()

                if idx == 4:
                    this_action = 'VOLUP'
                    if tv_st == 'ON':
                        vol_up()

                if idx == 5:
                    this_action = 'VOLDOWN'
                    if tv_st =='ON':
                        vol_down()
            
            if(sc_st == 'EX'):
                if idx == 6:
                    this_action = 'FANON'
                    fan_st = 'ON'
                    ser.write(b"i")

                if idx == 7:
                    this_action = 'FANOFF'
                    fan_st = 'OFF'
                    ser.write(b"j")
                    
            if idx == 8:
                this_action = 'CLICK'
  
            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255, 255, 255), thickness=2)
         
    cv2.imshow('Filter', img)
    if cv2.waitKey(1) == ord('q'):
        break